# LLMs 메모리의 이해

이전 노트에서는 OpenAI 모델이 Azure Cognitive Search의 결과를 어떻게 향상시킬 수 있는지 성공적으로 탐구했습니다.

하지만 아직 LLM과의 대화 방법을 찾지 못했습니다. 예를 들어 [Bing Chat](http://chat.bing.com/), 을 사용하면 이전 응답을 이해하고 참조할 수 있기 때문에 가능합니다.

아직 많은 사람들이 GPT모델이 기억력을 가지고 있다는 잘못된 인식이 있습니다. GPT모델을 학습이 되어 지식을 가지고 있지만, 질문한 이전의 정보는 가지고 있지 않습니다.

그렇기에 이 노트에서는 프롬프트와 컨텍스트를 사용하여 "메모리를 포함한 LLM을 효과적으로 처리"할 수 있는 방법을 설명하는 것이 목표입니다.

In [ ]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"]
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"]
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]
os.environ["OPENAI_API_TYPE"] = "azure"

In [ ]:
QUESTION = "Tell me some use cases for reinforcement learning?"
FOLLOW_UP_QUESTION = "Give me the main points of our conversation"

In [ ]:
# We create a very simple prompt template, just the question as is:
prompt = PromptTemplate(
    input_variables=["question"],
    template="{question}",
)

chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
#Now let's ask a follow up question
chain.run(FOLLOW_UP_QUESTION)

In [8]:
hist_prompt = PromptTemplate(
    input_variables=["history", "question"],
    template="""
                {history}
                Human: {question}
                AI:
            """
    )
chain = LLMChain(llm=llm, prompt=hist_prompt)

In [9]:
Conversation_history = """
Human: {question}
AI: {response}
""".format(question=QUESTION, response=response)

**Bingo!**, 이로써 우리는 LLM을 이용하여 챗봇을 만드는 방법을 알게 되었습니다. 이제 대화의 상태/이력을 유지하고 매번 컨텍스트로 전달시켜 보겠습니다. 

## 이제 GPT모델에 기억력을 추가하여 메모리의 개념을 이해했으므로 GPT Smart Search 엔진으로 돌아가 보겠습니다.

In [13]:
%%time

# Search in text-based indexes first and update vector indexes
k=10 # Top k results per each text-based index
ordered_results = get_search_results(QUESTION, text_indexes, k=k, reranker_threshold=1, vector_search=False)
update_vector_indexes(ordered_search_results=ordered_results, embedder=embedder)

# Search in all vector-based indexes available
similarity_k = 5 # top results from multi-vector-index similarity search
ordered_results = get_search_results(QUESTION, vector_indexes, k=k, vector_search=True,
                                        similarity_k=similarity_k,
                                        query_vector = embedder.embed_query(QUESTION))
print("Number of results:",len(ordered_results))

Number of results: 5
CPU times: user 528 ms, sys: 40.4 ms, total: 568 ms
Wall time: 3.02 s


In [14]:
# Uncomment the below line if you want to inspect the ordered results
# ordered_results

In [15]:
top_docs = []
for key,value in ordered_results.items():
    location = value["location"] if value["location"] is not None else ""
    top_docs.append(Document(page_content=value["chunk"], metadata={"source": location+os.environ['BLOB_SAS_TOKEN']}))
        
print("Number of chunks:",len(top_docs))

Number of chunks: 5


In [16]:
# Calculate number of tokens of our docs
if(len(top_docs)>0):
    tokens_limit = model_tokens_limit(MODEL) # this is a custom function we created in common/utils.py
    prompt_tokens = num_tokens_from_string(COMBINE_CHAT_PROMPT_TEMPLATE) # this is a custom function we created in common/utils.py
    context_tokens = num_tokens_from_docs(top_docs) # this is a custom function we created in common/utils.py
    
    requested_tokens = prompt_tokens + context_tokens + COMPLETION_TOKENS
    
    chain_type = "map_reduce" if requested_tokens > 0.9 * tokens_limit else "stuff"  
    
    print("System prompt token count:",prompt_tokens)
    print("Max Completion Token count:", COMPLETION_TOKENS)
    print("Combined docs (context) token count:",context_tokens)
    print("--------")
    print("Requested token count:",requested_tokens)
    print("Token limit for", MODEL, ":", tokens_limit)
    print("Chain Type selected:", chain_type)
        
else:
    print("NO RESULTS FROM AZURE SEARCH")

System prompt token count: 2464
Max Completion Token count: 1000
Combined docs (context) token count: 1019
--------
Requested token count: 4483
Token limit for gpt-35-turbo-16k : 16384
Chain Type selected: stuff


In [17]:
%%time
# Get the answer
response = get_answer(llm=llm, docs=top_docs, query=QUESTION, language="English", chain_type=chain_type)
printmd(response['output_text'])

Reinforcement learning can be applied in various use cases, including:
1. Learning prevention strategies for epidemics of infectious diseases, such as pandemic influenza, by automatically learning mitigation policies in complex epidemiological models with a large state space<sup><a href="https://arxiv.org/pdf/2003.13676v1.pdf?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D" target="_blank">[1]</a></sup>.
2. Learning sparse reward tasks efficiently by combining self-imitation learning with exploration bonuses, which enhances exploration by producing intrinsic rewards when the agent visits novel states<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7206262/?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D" target="_blank">[2]</a></sup>.
3. Personalized hybrid recommendation algorithms for music based on reinforcement learning, which consider the simulation of the interaction process to capture changes in listeners' preferences sensitively<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7206183/?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D" target="_blank">[3]</a></sup>.
4. Automatic feature engineering in machine learning projects, where a framework called CAFEM (Cross-data Automatic Feature Engineering Machine) is used to optimize feature transformation and improve learning performance<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7206177/?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D" target="_blank">[4]</a></sup>.
5. Job scheduling in data centers, where an Advantage Actor-Critic (A2C) deep reinforcement learning approach called A2cScheduler is used to automatically learn scheduling policies and achieve competitive scheduling performance<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7206316/?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D" target="_blank">[5]</a></sup>.

These references provide more details and information about the respective use cases.

CPU times: user 9.97 ms, sys: 0 ns, total: 9.97 ms
Wall time: 21.5 s


And if we ask the follow up question:

In [18]:
response = get_answer(llm=llm, docs=top_docs,  query=FOLLOW_UP_QUESTION, language="English", chain_type=chain_type)
printmd(response['output_text'])

The main points of our conversation are about reinforcement learning in various domains. We discussed the use of deep reinforcement learning to learn prevention strategies in the context of pandemic influenza<sup><a href="https://arxiv.org/pdf/2003.13676v1.pdf?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D" target="_blank">[1]</a></sup>. We also talked about the Explore-then-Exploit framework, which combines self-imitation learning with exploration bonuses to improve performance in sparse reward tasks<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7206262/?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D" target="_blank">[2]</a></sup>. Additionally, we discussed the use of reinforcement learning in personalized music recommendation systems<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7206183/?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D" target="_blank">[3]</a></sup>. We also touched upon the topic of automatic feature engineering using a framework called CAFEM<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7206177/?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D" target="_blank">[4]</a></sup>. Lastly, we discussed the A2cScheduler, an Advantage Actor-Critic deep reinforcement learning approach for job scheduling in data centers<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7206316/?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D" target="_blank">[5]</a></sup>.

Anything else I can help you with?

지금까지 우리는 노트북 03과 동일한 메모리 없이 OpenAI 모델로 향상된 Azure Search의 결과를 받았습니다. 이제 메모리를 추가를 해보겠습니다.

In [19]:
# 입력/결과를 추적하고 대화를 진행하기 위해 필요한 메모리 객체
memory = ConversationBufferMemory(memory_key="chat_history",input_key="question")

response = get_answer(llm=llm, docs=top_docs, query=QUESTION, language="English", chain_type=chain_type, 
                        memory=memory)
printmd(response['output_text'])

Reinforcement learning has various use cases across different domains. Here are some examples:

1. **Epidemic prevention strategies**: Reinforcement learning can be used to automatically learn prevention strategies for infectious diseases. For example, a study used deep reinforcement learning to learn mitigation policies in complex epidemiological models with a large state space<sup><a href="https://arxiv.org/pdf/2003.13676v1.pdf?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D" target="_blank">[1]</a></sup>.

2. **Sparse reward tasks**: Reinforcement learning can be challenging when dealing with tasks that have sparse rewards. Self-imitation learning and exploration bonuses are two approaches that can address this challenge. A recent framework called Explore-then-Exploit (EE) interleaves self-imitation learning with an exploration bonus to enhance both exploitation and exploration in learning tasks<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7206262/?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D" target="_blank">[2]</a></sup>.

3. **Personalized recommendation systems**: Reinforcement learning can be applied to personalized recommendation systems. For example, a personalized hybrid recommendation algorithm for music based on reinforcement learning was proposed. It uses techniques like weighted matrix factorization and convolutional neural networks to learn and extract song feature vectors, and it continuously updates the model based on the preferences of listeners for songs and song transitions<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7206183/?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D" target="_blank">[3]</a></sup>.

4. **Automatic feature engineering**: Feature engineering is a crucial task in machine learning projects. Reinforcement learning can be used to automate feature engineering. For example, a framework called Cross-data Automatic Feature Engineering Machine (CAFEM) formalizes the feature engineering problem as an optimization problem over a Feature Transformation Graph (FTG). It includes a feature engineering learner that learns fine-grained strategies on a single dataset and a cross-data component that speeds up feature engineering learning on unseen datasets<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7206177/?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D" target="_blank">[4]</a></sup>.

5. **Job scheduling in data centers**: Reinforcement learning can be applied to job scheduling in data centers. For example, an Advantage Actor-Critic (A2C) deep reinforcement learning-based approach called A2cScheduler was proposed for job scheduling. It consists of two agents, an actor and a critic, that work together to learn the scheduling policy and reduce estimation errors. The approach showed competitive performance using both simulated workloads and real data from an academic data center<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7206316/?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X

In [20]:
# 후속질문 추가:
response = get_answer(llm=llm, docs=top_docs, query=FOLLOW_UP_QUESTION, language="English", chain_type=chain_type, 
                      memory=memory)
printmd(response['output_text'])

Here are the main points of our conversation:

1. Reinforcement learning has various use cases across different domains.
2. Some examples of use cases for reinforcement learning include epidemic prevention strategies, sparse reward tasks, personalized recommendation systems, automatic feature engineering, and job scheduling in data centers.

Would you like more information about any of these use cases?

In [21]:
# 후속질문 추가２：
response = get_answer(llm=llm, docs=top_docs, query="Thank you", language="English", chain_type=chain_type,  
                      memory=memory)
printmd(response['output_text'])

The main points of our conversation are as follows:

1. Reinforcement learning has various use cases across different domains.
2. Some examples of use cases for reinforcement learning include:
   - Epidemic prevention strategies, where deep reinforcement learning is used to learn prevention strategies for infectious diseases<sup><a href="https://arxiv.org/pdf/2003.13676v1.pdf?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D" target="_blank">[1]</a></sup>.
   - Sparse reward tasks, where self-imitation learning and exploration bonuses can be used to address the challenge of sparse rewards<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7206262/?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D" target="_blank">[2]</a></sup>.
   - Personalized recommendation systems, where reinforcement learning can be applied to recommend personalized song sequences<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7206183/?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D" target="_blank">[3]</a></sup>.
   - Automatic feature engineering, where reinforcement learning can be used to automate the process of feature engineering<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7206177/?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D" target="_blank">[4]</a></sup>.
   - Job scheduling in data centers, where reinforcement learning can be applied to optimize job scheduling and resource allocation<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7206316/?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D" target="_blank">[5]</a></sup>.

Please let me know if you would like more information about any of these use cases.

You might get a different answer on the above cell, and it is ok, this bot is not yet well configured to answer any question that is not related to its knowledge base, including salutations.

Let's check our memory to see that it's keeping the conversation

In [22]:
memory.buffer

'Human: Tell me some use cases for reinforcement learning?\nAI: Reinforcement learning has various use cases across different domains. Here are some examples:\n\n1. **Epidemic prevention strategies**: Reinforcement learning can be used to automatically learn prevention strategies for infectious diseases. For example, a study used deep reinforcement learning to learn mitigation policies in complex epidemiological models with a large state space<sup><a href="https://arxiv.org/pdf/2003.13676v1.pdf?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D" target="_blank">[1]</a></sup>.\n\n2. **Sparse reward tasks**: Reinforcement learning can be challenging when dealing with tasks that have sparse rewards. Self-imitation learning and exploration bonuses are two approaches that can address this challenge. A recent framework called Explore-then-Exploit (EE) interleaves self-imitation learning with an e

## CosmosDB를 영구 메모리로 사용

이전 셀에서는 로컬 RAM 메모리를 챗봇에 추가한 적이 있는데, 지속성은 없지만 앱 사용자의 세션이 종료되면 삭제되었습니다. 
그렇기에 분석 및 감사뿐만 아니라 권장 사항을 제공하고자 할 때에도  봇 사용자 대화의 지속적인 저장을 위해 데이터베이스를 사용하는 것을 추천 드립니다. 

이번 Hands on Labs에서도 GPT모델의 대화 기억을 위해 LangChain에서 제공하는 CosmosDBChatMessageHistory 기능과 CosmosDB을 사용하겠습니다. 
더 자세한 내용은 [여기](https://python.langchain.com/en/latest/_modules/langchain/memory/chat_message_histories/cosmos_db.html)에서 확인하실 수 있습니다. 

In [23]:
# Create CosmosDB instance from langchain cosmos class.
cosmos = CosmosDBChatMessageHistory(
    cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
    cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
    cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
    connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
    session_id="Agent-Test-Session" + str(random.randint(1, 1000)),
    user_id="Agent-Test-User" + str(random.randint(1, 1000))
    )

# prepare the cosmosdb instance
cosmos.prepare_cosmos()

In [24]:
# Create or Memory Object
memory = ConversationBufferMemory(memory_key="chat_history",input_key="question",chat_memory=cosmos)

In [25]:
# Testing using our Question
response = get_answer(llm=llm, docs=top_docs, query=QUESTION, language="English", chain_type=chain_type, 
                        memory=memory)
printmd(response['output_text'])

Reinforcement learning has various use cases in different domains. Here are some examples:

1. **Epidemic Prevention**: Reinforcement learning can be used to automatically learn prevention strategies for infectious diseases. For example, a study used deep reinforcement learning to learn mitigation policies in complex epidemiological models with a large state space<sup><a href="https://arxiv.org/pdf/2003.13676v1.pdf?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D">[1]</a></sup>.

2. **Sparse Reward Tasks**: Reinforcement learning is used to tackle tasks with sparse rewards, where efficient exploitation and exploration are required. One approach is self-imitation learning, which encourages exploitation by imitating past good trajectories. Another approach is exploration bonuses, which enhance exploration by providing intrinsic rewards. A novel framework called Explore-then-Exploit (EE) has been introduced, which interleaves self-imitation learning with an exploration bonus to strengthen the effect of these two algorithms<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7206262/?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D">[2]</a></sup>.

3. **Personalized Recommendation Systems**: Reinforcement learning can be used to improve personalized recommendation systems. For example, a personalized hybrid recommendation algorithm for music based on reinforcement learning was proposed. It recommends song sequences that match listeners' preferences better by simulating the interaction process and updating the model continuously based on their preferences<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7206183/?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D">[3]</a></sup>.

4. **Feature Engineering**: Reinforcement learning can be used to automate feature engineering, which is a time-consuming and challenging task in machine learning projects. A framework called Cross-data Automatic Feature Engineering Machine (CAFEM) has been proposed, which formalizes the feature engineering problem as an optimization problem and learns fine-grained feature engineering strategies using reinforcement learning<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7206177/?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D">[4]</a></sup>.

5. **Job Scheduling**: Reinforcement learning can be used for efficient job scheduling in data centers. An approach called A2cScheduler, based on Advantage Actor-Critic (A2C) deep reinforcement learning, has been proposed for job scheduling. It consists of two agents, the actor and the critic, which learn the scheduling policy and reduce estimation error, respectively<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7206316/?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D">[5]</a></sup>.

These are just a few examples of the use cases for reinforcement learning. It

In [26]:
# Now we add a follow up question:
response = get_answer(llm=llm, docs=top_docs, query=FOLLOW_UP_QUESTION, language="English", chain_type=chain_type, 
                      memory=memory)
printmd(response['output_text'])

Here are the main points of our conversation:

1. Reinforcement learning has various use cases in different domains.
2. One use case is epidemic prevention, where deep reinforcement learning is used to learn prevention strategies for infectious diseases<sup><a href="https://arxiv.org/pdf/2003.13676v1.pdf?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D">[1]</a></sup>.
3. Another use case is tackling tasks with sparse rewards, where self-imitation learning and exploration bonuses are used to enhance exploitation and exploration<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7206262/?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D">[2]</a></sup>.
4. Reinforcement learning can be used to improve personalized recommendation systems, such as a hybrid recommendation algorithm for music based on reinforcement learning<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7206183/?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D">[3]</a></sup>.
5. Reinforcement learning can automate feature engineering tasks, such as a framework called CAFEM that learns fine-grained feature engineering strategies using reinforcement learning<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7206177/?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D">[4]</a></sup>.
6. Reinforcement learning can be used for efficient job scheduling in data centers, such as the A2cScheduler approach based on Advantage Actor-Critic (A2C) deep reinforcement learning<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7206316/?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D">[5]</a></sup>.

These points summarize the main use cases of reinforcement learning that we discussed. Is there anything else I can help you with?

In [27]:
# Another follow up query
response = get_answer(llm=llm, docs=top_docs, query="Thank you", language="English", chain_type=chain_type,  
                      memory=memory)
printmd(response['output_text'])

Based on our conversation, here are the main points:

1. Reinforcement learning has various use cases in different domains.
2. One use case is epidemic prevention, where deep reinforcement learning is used to learn prevention strategies for infectious diseases<sup><a href="https://arxiv.org/pdf/2003.13676v1.pdf?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D">[1]</a></sup>.
3. Another use case is tackling tasks with sparse rewards, where self-imitation learning and exploration bonuses are used to enhance exploitation and exploration<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7206262/?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D">[2]</a></sup>.
4. Reinforcement learning can be used to improve personalized recommendation systems, such as a hybrid recommendation algorithm for music based on reinforcement learning<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7206183/?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D">[3]</a></sup>.
5. Reinforcement learning can automate feature engineering tasks, such as a framework called CAFEM that learns fine-grained feature engineering strategies using reinforcement learning<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7206177/?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D">[4]</a></sup>.
6. Reinforcement learning can be used for efficient job scheduling in data centers, such as the A2cScheduler approach based on Advantage Actor-Critic (A2C) deep reinforcement learning<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7206316/?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D">[5]</a></sup>.

These points summarize the main use cases of reinforcement learning that we discussed. Let me know if there's anything else I can assist you with.

Azure Cosmos DB를 통해 전체 대화 내용을 확인해 보겠습니다


In [28]:
#load message from cosmosdb
cosmos.load_messages()
cosmos.messages

[HumanMessage(content='Tell me some use cases for reinforcement learning?', additional_kwargs={}, example=False),
 AIMessage(content='Reinforcement learning has various use cases in different domains. Here are some examples:\n\n1. **Epidemic Prevention**: Reinforcement learning can be used to automatically learn prevention strategies for infectious diseases. For example, a study used deep reinforcement learning to learn mitigation policies in complex epidemiological models with a large state space<sup><a href="https://arxiv.org/pdf/2003.13676v1.pdf?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D">[1]</a></sup>.\n\n2. **Sparse Reward Tasks**: Reinforcement learning is used to tackle tasks with sparse rewards, where efficient exploitation and exploration are required. One approach is self-imitation learning, which encourages exploitation by imitating past good trajectories. Another approac

![CosmosDB Memory](https://raw.githubusercontent.com/MSUSAzureAccelerators/Azure-Cognitive-Search-Azure-OpenAI-Accelerator/69114f93a7c245fd79009f01169e746859776d77//images/cosmos-chathistory.png)

# Summary
##### 애플리케이션에 메모리를 추가하면 이전 대화를 기반으로 GPT모델과 사용자가 대화할 수 있지만, 이 기능은 LLM과 함께 제공되는 것이 아니라 질문을 컨텍스트 형태로 LLM에 제공해야 하는 것입니다.

그렇기에 우리는 Cosmos DB를 이용하여 영구적인 memory를 추가하였습니다. 


## <u>Important Note</u>:<br>
모든 코드가 GPT-3.5 모델과 호환되는 상태로 유지되지만 GPT-4로 전환할 것을 강력히 권장합니다. 그 이유는 다음과 같습니다:

**GPT-3.5-Turbo**는 7세 어린아이에 비유할 수 있습니다. 간결한 설명을 제공할 수 있지만 정확한 설명을 따라하기가 어려운 경우가 많습니다. 또한 기억력이 부족하여 지속적인 대화가 어려울 수 있습니다.

**GPT-3.5-Turbo-16k**는 같은 7살짜리 아이와 닮았지만, 좀 더 긴 지시에 대해 이해가 가능합니다. 하지만, 여전히 절반 정도는  정확하게 이해하는데 어려움을 겪고 있습니다.

**GPT-4**는 10-12세 어린이의 능력을 보여줍니다. 향상된 추론 능력을 가지고 있으며 Prompt를 통한 명령을 더욱 준수합니다. 명령에 대한 기억 유지는 중간 정도이지만 명령을 따르는 것에 탁월합니다.

**GPT-4-32k**는 기억력이 뛰어난 10-12세 어린이와 유사하며, 긴 명령어를 이해하고 의미 있는 대화를 나눌 수 있으며, 기억력이 뛰어나 세밀한 반응을 얻을 수 있습니다.

